In [1]:
# test derivatives
import numpy as np
%matplotlib
from matplotlib import pyplot as plt
import os,sys
import scipy as sp


Using matplotlib backend: MacOSX


In [2]:
import ewc
import wrm_vangenuchten
import permafrost_model_explicit_fpd
import permafrost_model_explicit_fpd_smoothed

In [4]:
def plot(wrm, axs, style, color=None, control=False):
    sats = np.concatenate([np.linspace(0.21,0.9,1000), np.linspace(0.9,1,1000)])
    
    try:
        pcs = np.array([wrm.capillaryPressure(s) for s in sats])
    except RuntimeError:
        pcs = np.array([wrm._vg.capillaryPressure(s) for s in sats])
        
    sats = np.array([wrm.saturation(pc) for pc in pcs])
    dsats = np.array([wrm.d_saturation(pc) for pc in pcs])

    #print "Test pcs:", pcs[-10:]
    #print "Test sats:", sats[-10:]
    #print "Test dsats:", dsats[-10:]
    
    if control:
        print "auto x = std::vector<double>{" + ",".join(["%8.16e"%x for x in wrm._control_pcs])+"};"
        print "auto y = std::vector<double>{" + ",".join(["%8.16e"%x for x in wrm._control_sats])+"};"
        print "auto dy = std::vector<double>{" + ",".join(["%8.16e"%x for x in wrm._control_dsats])+"};"
        print "auto test_x = std::vector<double>{" + ",".join(["%8.16e"%x for x in pcs])+"};"
        print "auto test_y = std::vector<double>{" + ",".join(["%8.16e"%x for x in sats])+"};"
        print "auto test_dy = std::vector<double>{" + ",".join(["%8.16e"%x for x in dsats])+"};"



    kr = np.array([wrm.k_relative(s) for s in sats])
    dkrs = np.array([wrm.d_k_relative(s) for s in sats])
    
    axs[0].semilogy(sats,pcs, style, color=color, label=wrm.label())
    axs[1].plot(pcs, sats, style, color=color, label=wrm.label())
    axs[2].plot(pcs, dsats, style, color=color, label=wrm.label())
               
    axs[3].plot(sats,kr, style, color=color, label=wrm.label())
    axs[4].plot(pcs, kr, style, color=color, label=wrm.label()) 
    axs[5].plot(pcs,dkrs, style, color=color, label=wrm.label() )

    print "Test kr:", kr[-10:]
    print "test dkr:", dkrs[-10:]
    
    if control:
        axs[0].semilogy(wrm._control_sats, wrm._control_pcs, 'x', color=color)
        axs[1].plot(wrm._control_pcs, wrm._control_sats, 'x', color=color)
        axs[2].plot(wrm._control_pcs, wrm._control_dsats, 'x', color=color)
        axs[3].plot(wrm._control_sats, wrm._control_krs, 'x', color=color)
        axs[4].plot(wrm._control_pcs, wrm._control_krs, 'x', color=color)
        axs[5].plot(wrm._control_pcs, wrm._control_dkrs, 'x', color=color)

    
def get_ax():
    fig,ax = plt.subplots(2,3, figsize=(12,6))
    return fig,ax.flatten()

def decorate(axs):
    axs[0].set_xlabel('saturation [-]')
    axs[0].set_ylabel('log capillary pressure [Pa]')

    axs[1].set_xlabel('capillary presure [Pa]')
    axs[1].set_ylabel('saturation [-]')
  
    axs[2].set_xlabel('capillary pressure [Pa]')
    axs[2].set_ylabel("dsaturation/dpc")
    
    axs[3].set_xlabel('saturation [-]')
    axs[3].set_ylabel('rel perm [-]')    

    axs[4].set_xlabel('capillary pressure [Pa]')
    axs[4].set_ylabel('rel perm [-]')

    axs[5].set_xlabel('capillary pressure [Pa]')
    axs[5].set_ylabel('drel perm [-]')

        
    axs[0].set_xlim(0,1.05)
    axs[1].set_xlim(0,10000)
    axs[2].set_xlim(0,3.e3)

    axs[3].set_xlim(0,1)
    axs[4].set_xlim(0,10000)
    axs[5].set_xlim(0,3.e3)
    
    axs[3].legend()

    

In [ ]:
wrm0 = wrm_vangenuchten.VanGenuchten(alpha=0.00056, m=0.24, sr=0.18, smoothing_interval=0.1)
#wrm1 = wrm_vangenuchten.VanGenuchten(alpha=0.0056, m=0.24, sr=0.18, smoothing_interval=0.)
#wrm4 = wrm_vangenuchten.VanGenuchten(alpha=0.00056, m=0.39, sr=0.18, smoothing_interval=0.)
#wrm2 = wrm_vangenuchten.VanGenuchten(alpha=0.00056, m=0.54, sr=0.18, smoothing_interval=0.)
wrm3 = wrm_vangenuchten.VanGenuchten(alpha=0.00056, m=0.74, sr=0.18, smoothing_interval=0.)
wrm2 = wrm_vangenuchten.VanGenuchten(alpha=0.00056, m=0.24, sr=0.18, smoothing_interval=0.1, smoothing_interval_sat=1./0.00056)

f,a = get_ax()

#plot(wrm1, a, '-', 'r')
#plot(wrm4, a, '-', 'b')
plot(wrm3, a, '-', 'c')
plot(wrm0, a, '-',color='k')
plot(wrm2, a, '-', color='violet')



#sats = np.concatenate([np.linspace(0.21,0.9,1000), np.linspace(0.9,1,1000)])
#pcs = np.array([wrm0.capillaryPressure(s) for s in sats])
#kr = np.array([wrm3b.k_relative(s) for s in sats])
    
#a[0].semilogy(sats,pcs, 'm')
#a[1].plot(pcs, sats, 'm')
#a[2].plot(pcs, kr, 'm',label='franken')
    
#pcs_m = (pcs[1:] + pcs[:-1])/2.
#dkrs = (kr[1:] - kr[:-1])/(pcs[1:] - pcs[:-1])
#a[3].plot(pcs_m,dkrs, 'm')



decorate(a)
plt.show()

In [ ]:
            <Parameter name="van Genuchten alpha" type="double" value="0.00056" />
            <Parameter name="van Genuchten m" type="double" value="0.24" />
            <Parameter name="residual saturation" type="double" value="0.18" />
            <Parameter name="smoothing interval width [saturation]" type="double" value="0.01" />
            
            
            

In [ ]:
wrm0 = wrm_vangenuchten.VanGenuchten(alpha=0.00056, m=0.24, sr=0.18, smoothing_interval=0.)
wrm1 = wrm_vangenuchten.VanGenuchten(alpha=0.00056, m=0.24, sr=0.18, smoothing_interval=0.01)
wrm2 = wrm_vangenuchten.VanGenuchten(alpha=0.00056, m=0.24, sr=0.18, smoothing_interval=0.001)
wrm3 = wrm_vangenuchten.VanGenuchten(alpha=0.00056, m=0.24, sr=0.18, smoothing_interval=0.1)

f,a = get_ax()

plot(wrm0, a, '-',color='g')
plot(wrm1, a, '-',color='r')
plot(wrm2, a, '-',color='b')
plot(wrm3, a, '-',color='k')
decorate(a)
plt.show()

In [ ]:
pmC = permafrost_model_explicit_fpd_smoothed.PermafrostModel(wrmC)
test(pmC)

In [14]:
f,a = get_ax()

print "Plotting WRM0"
print "---------------------"
wrm0 = wrm_vangenuchten.VanGenuchten(alpha=0.00056, m=0.74, sr=0.18, smoothing_interval=0.0)
plot(wrm0, a, '-',color='c')

print "Plotting WRM1"
print "---------------------"
wrm1 = wrm_vangenuchten.VanGenuchtenSpline(alpha=0.00056, m=0.74, sr=0.18, smoothing_interval=0.0, n_control=20)
plot(wrm1, a, '-',color='r', control=True)
decorate(a)
plt.show()

Plotting WRM0
---------------------
Test kr: [ 0.98338184  0.98477803  0.98621966  0.98771501  0.98927561  0.9909184
  0.99267044  0.99458108  0.9967656   1.        ]
test dkr: [ 13.7430787   14.1631008   14.654253    15.24181747  15.96651786
  16.89974391  18.1830303   20.15847398  24.04809942   0.        ]
Plotting WRM1
---------------------
Control sats: [ 1.          0.99440813  0.97778507  0.95058424  0.91354761  0.86768544
  0.81424874  0.75469512  0.69064905  0.62385753  0.55614247  0.48935095
  0.42530488  0.36575126  0.31231456  0.26645239  0.22941576  0.20221493
  0.18559187]
Control PCs: [     0.            529.04601361    762.07344967    948.1815387
   1113.32167442   1268.58434156   1420.6851918    1574.78946324
   1735.72001171   1908.74257205   2100.34484612   2319.30159285
   2578.43843335   2897.93782333   3312.25230604   3886.52283183
   4762.96497638   6332.49217544  10299.0174332 ]
Control dsats: [  0.00000000e+00  -4.03270837e-05  -1.08558208e-04  -1.86347054e-04
 

In [5]:
f,a = get_ax()

print "Plotting WRM3"
print "---------------------"
wrm3 = wrm_vangenuchten.VanGenuchten(alpha=0.00056, m=0.24, sr=0.18, smoothing_interval=0.0)
plot(wrm3, a, '-',color='m')

print "Plotting WRM4"
print "---------------------"
wrm4 = wrm_vangenuchten.VanGenuchtenSpline(alpha=0.00056, m=0.24, sr=0.18, smoothing_interval=0.0, n_control=20)
plot(wrm4, a, '-',color='r', control=True)


print "Plotting WRM5"
print "---------------------"
wrm5 = wrm_vangenuchten.VanGenuchten(alpha=0.00056, m=0.24, sr=0.18, smoothing_interval=0.1)
plot(wrm5, a, '-',color='c')

print "Plotting WRM6"
print "---------------------"
wrm6 = wrm_vangenuchten.VanGenuchtenSpline(alpha=0.00056, m=0.24, sr=0.18, smoothing_interval=0.1, n_control=20)
plot(wrm6, a, '-',color='b', control=True)


decorate(a)
plt.show()

Plotting WRM3
---------------------
Test pcs: [ 29.84317413  27.28128629  24.64261148  21.91305637  19.07308809
  16.09404568  12.93027121   9.49891806   5.60773171   0.        ]
Test sats: [ 0.9990991  0.9991992  0.9992993  0.9993994  0.9994995  0.9995996
  0.9996997  0.9997998  0.9998999  1.       ]
Test dsats: [ -3.96082341e-05  -3.85256657e-05  -3.73313190e-05  -3.59954268e-05
  -3.44734718e-05  -3.26939797e-05  -3.05297770e-05  -2.77142317e-05
  -2.34799264e-05   0.00000000e+00]
Test kr: [ 0.52621428  0.53738014  0.54979882  0.56381651  0.57995441  0.59905564
  0.62263348  0.65389434  0.7021992   1.        ]
test dkr: [ 106.16005455  117.33125963  131.35941422  149.55973619  174.22908495
  209.7976477   266.14428012  371.14889275  651.24358885    0.        ]
Plotting WRM4
---------------------
Control sats: [ 1.          0.99440813  0.97778507  0.95058424  0.91354761  0.86768544
  0.81424874  0.75469512  0.69064905  0.62385753  0.55614247  0.48935095
  0.42530488  0.36575126  0.31

In [8]:
f,a = get_ax()

print "Plotting WRM7"
print "---------------------"
wrm7 = wrm_vangenuchten.VanGenuchten(alpha=0.00056, m=0.24, sr=0.18, smoothing_interval=0.1)
plot(wrm7, a, '-',color='k')

print "Plotting WRM8"
print "---------------------"
wrm8 = wrm_vangenuchten.VanGenuchtenSpline(alpha=0.00056, m=0.24, sr=0.18, smoothing_interval=0.1, n_control=80)
plot(wrm8, a, '-',color='r', control=True)

print "Plotting WRM9"
print "---------------------"
wrm9 = wrm_vangenuchten.VanGenuchtenSpline(alpha=0.00056, m=0.24, sr=0.18, smoothing_interval=0.1, n_control=40)
plot(wrm9, a, '-',color='g', control=True)


print "Plotting WRM10"
print "---------------------"
wrm10 = wrm_vangenuchten.VanGenuchtenSpline(alpha=0.00056, m=0.24, sr=0.18, smoothing_interval=0.1, n_control=20)
plot(wrm10, a, '-',color='b', control=True)

decorate(a)
plt.show()

Plotting WRM7
---------------------
Test pcs: [ 29.84317413  27.28128629  24.64261148  21.91305637  19.07308809
  16.09404568  12.93027121   9.49891806   5.60773171   0.        ]
Test sats: [ 0.9990991  0.9991992  0.9992993  0.9993994  0.9994995  0.9995996
  0.9996997  0.9997998  0.9998999  1.       ]
Test dsats: [ -3.96082341e-05  -3.85256657e-05  -3.73313190e-05  -3.59954268e-05
  -3.44734718e-05  -3.26939797e-05  -3.05297770e-05  -2.77142317e-05
  -2.34799264e-05   0.00000000e+00]
Test kr: [ 0.9997707   0.9998187   0.9998611   0.99989788  0.99992904  0.99995455
  0.99997442  0.99998862  0.99999715  1.        ]
test dkr: [ 0.50753314  0.4515916   0.3955373   0.33937024  0.28309043  0.22669785
  0.17019253  0.11357444  0.0568436   0.        ]
Plotting WRM8
---------------------
Control sats: [ 1.          0.99967585  0.99870392  0.99708575  0.99482389  0.99192192
  0.98838443  0.98421701  0.97942625  0.97401973  0.968006    0.96139456
  0.95419587  0.94642131  0.93808318  0.92919465  

In [ ]:
1.0/0.00056


In [ ]:
import spline
a = spline.Spline(0,1,0,529.0460136,0.9944081,-4.03270837e-05)
a.Value(150)

In [ ]:
print wrm1.d_saturation(0.)

In [5]:
f,a = get_ax()
print "Plotting WRM9"
print "---------------------"
wrm9 = wrm_vangenuchten.VanGenuchtenSpline(alpha=1.36e-4, n=1.39, sr=0.)
plot(wrm9, a, '-',color='g', control=True)
decorate(a)
plt.show()

Plotting WRM9
---------------------
Control sats: [  1.00000000e+00   9.99748271e-01   9.98993338e-01   9.97735961e-01
   9.95977406e-01   9.93719444e-01   9.90964349e-01   9.87714893e-01
   9.83974351e-01   9.79746487e-01   9.75035559e-01   9.69846310e-01
   9.64183967e-01   9.58054229e-01   9.51463269e-01   9.44417724e-01
   9.36924689e-01   9.28991707e-01   9.20626766e-01   9.11838291e-01
   9.02635129e-01   8.93026547e-01   8.83022222e-01   8.72632225e-01
   8.61867019e-01   8.50737444e-01   8.39254706e-01   8.27430367e-01
   8.15276334e-01   8.02804844e-01   7.90028455e-01   7.76960032e-01
   7.63612734e-01   7.50000000e-01   7.36135537e-01   7.22033306e-01
   7.07707507e-01   6.93172563e-01   6.78443111e-01   6.63533982e-01
   6.48460188e-01   6.33236907e-01   6.17879468e-01   6.02403334e-01
   5.86824089e-01   5.71157419e-01   5.55419100e-01   5.39624978e-01
   5.23790958e-01   5.07932982e-01   4.92067018e-01   4.76209042e-01
   4.60375022e-01   4.44580900e-01   4.28842581e-01  